# Python Network Programming Cookbook Second Edition
[source code](https://github.com/PacktPublishing/Python-Network-Programming-Cookbook-Second-Edition)

# 1. Get local machine info

In [1]:
import socket
hostname = socket.gethostname()
ipv4addr = socket.gethostbyname(hostname)
print(f"hostname: {hostname}; ipv4 address: {ipv4addr}")

hostname: kaifage; ipv4 address: 127.0.1.1


# 2. Get a remote machine's ip address

In [13]:
remoteHost = 'google.com' # try a non-existant host name
remoteIP = ''
try: 
    remoteIP = socket.gethostbyname(remoteHost)
    print(f"remote host {remoteHost} has ip address {remoteIP}")
except socket.error as errMsg:
    print(f"{remoteHost}: {errMsg}")

googlex.com: [Errno -5] No address associated with hostname


# 3. IPv4 address conversion

In [16]:
from binascii import hexlify
for ip_addr in ['127.0.0.1', '192.168.0.1']:
    packed_ip_addr = socket.inet_aton(ip_addr)
    unpacked_ip_addr = socket.inet_ntoa(packed_ip_addr)
    print ("IP Address: %s => Packed: %s, Unpacked: %s" %(ip_addr, hexlify(packed_ip_addr), unpacked_ip_addr))

IP Address: 127.0.0.1 => Packed: b'7f000001', Unpacked: 127.0.0.1
IP Address: 192.168.0.1 => Packed: b'c0a80001', Unpacked: 192.168.0.1


# 4. Find service name
Compare the output with [List of TCP and UDP port numbers](https://en.wikipedia.org/wiki/List_of_TCP_and_UDP_port_numbers)

In [ ]:
for port in range(1025):
    try:
        print ("TCP port: %d => service name: %s" %(port, socket.getservbyport(port, 'tcp')))
        print ("UDP port: %d => service name: %s" %(port, socket.getservbyport(port, 'udp')))
    except socket.error as errMsg:
        print(f"port {port}: {errMsg}")


# 5. Convert integers between host byte order and network byte order

In [6]:
# 32-bit
longdata = 0x12345678
print ("Long integer: %x => host byte order: %x, Network byte order: %x" %(longdata, socket.ntohl(longdata), socket.htonl(longdata)))
# 16-bit
shortdata = 0xabcd
print ("short integer: %x => host byte order: %x, Network byte order: %x" %(shortdata, socket.ntohs(shortdata), socket.htons(shortdata)))

Long integer: 12345678 => host byte order: 78563412, Network byte order: 78563412
short integer: abcd => host byte order: cdab, Network byte order: cdab


# 6. Get and set socket timeout

In [11]:
s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
print ("Default socket timeout: %s" %s.gettimeout())
s.settimeout(100) # in seonds, find the usage: s.settimeout?
print ("Current socket timeout: %s seconds" %s.gettimeout())  

Default socket timeout: None
Current socket timeout: 100.0 seconds


# 7. Handle socket errors

# 8. Change socket buffer sizes

In [14]:
SEND_BUF_SIZE = 4096
RECV_BUF_SIZE = 4096

sock = socket.socket( socket.AF_INET, socket.SOCK_STREAM )

# Get the size of the socket's send buffer
bufsize = sock.getsockopt(socket.SOL_SOCKET, socket.SO_SNDBUF)
print ("[Before] send buffer size:%d" %bufsize)
# Get the size of the socket's receive buffer
bufsize = sock.getsockopt(socket.SOL_SOCKET, socket.SO_SNDBUF)
print ("[Before] receive buffer size:%d" %bufsize)

sock.setsockopt(socket.SOL_TCP, socket.TCP_NODELAY, 1)
sock.setsockopt(
        socket.SOL_SOCKET,
        socket.SO_SNDBUF,
        SEND_BUF_SIZE)
sock.setsockopt(
        socket.SOL_SOCKET,
        socket.SO_RCVBUF,
        RECV_BUF_SIZE)

bufsize = sock.getsockopt(socket.SOL_SOCKET, socket.SO_SNDBUF)
print ("[After] send buffer size:%d" %bufsize)
bufsize = sock.getsockopt(socket.SOL_SOCKET, socket.SO_SNDBUF)
print ("[After] receive buffer size:%d" %bufsize)

[Before] send buffer size:16384
[Before] receive buffer size:16384
[After] send buffer size:8192
[After] receive buffer size:8192


# 9. Blocking mode vs. non-blocking mode

In [ ]:
s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
s.setblocking(1)
s.settimeout(0.5)
s.bind(("127.0.0.1", 0))

socket_address = s.getsockname()
print ("Trivial Server launched on socket: %s" %str(socket_address))
while(1):
    s.listen(1)

# 10. Reuse socket address

# 11. Get internet time with NTP
[Network time protocol (NTP)](https://en.wikipedia.org/wiki/Network_Time_Protocol)

[List of Top Public Time Servers ](https://gist.github.com/mutin-sa/eea1c396b1e610a2da1e5550d94b0453)

In [3]:
import ntplib
from time import ctime

ntp_client = ntplib.NTPClient()
response = ntp_client.request('time.google.com')
#response = ntp_client.request('ntp-b.nist.gov')
print (ctime(response.tx_time))

Sat Aug 26 20:02:59 2023


# 12. Get internet time with SNTP
[Simple Network Time Protocol (SNTP)](https://www.advtimesync.com/online-manual/simple-network-time-protocol)

[Epoch time](http://www.epochconverter.com/)

In [5]:
import socket
import struct
import sys
import time

NTP_SERVER = "0.uk.pool.ntp.org"
TIME1970 = 2208988800 # reference epoch for 1970/1/1


client = socket.socket( socket.AF_INET, socket.SOCK_DGRAM )
data = '\x1b' + 47 * '\0'
client.sendto( data.encode('utf-8'), ( NTP_SERVER, 123 ))
data, address = client.recvfrom( 1024 )
if data:
    print ('Response received from:', address)
t = struct.unpack( '!12I', data )[10]
t -= TIME1970
print ('\tTime=%s' % time.ctime(t))

Response received from: ('152.67.150.30', 123)
	Time=Sat Aug 26 20:26:22 2023
